In [ ]:
'''from google.colab import files

uploaded = files.upload()
deltas, indexes = deltas_in_search('search_J1J-1')
print(deltas)

print(len(deltas))
counter = 1
for k,v in deltas.items():
  counter *= len(v)
  
print(counter)
'''

In [ ]:
print(len(data))T
print(data[-10:])

# Write output csv file

In [ ]:
from google.colab import files

#import file csv
uploaded = files.upload()


In [ ]:
import csv
import random
random.seed(227)

file_csv = open('test.csv', 'w', newline='')

filewriter = csv.writer(file_csv, delimiter=',',
                            quotechar='|', quoting=csv.QUOTE_MINIMAL)
filewriter.writerow([1,2,3,4,5,6])
for i in range(int(len(data)/2), len(data)):
    filewriter.writerow(data[i])

#files.download('train.csv')

In [ ]:
file_csv.close()
files.download('test.csv')


In [1]:
from keras.models import Model
from keras.layers import Dense, Input, GRU


def model_GRU(input_shape, Ty):
    i = Input(shape=input_shape, dtype='float32')
    X = GRU(16, return_sequences=False)(i)
    #X = LSTM(16, return_sequences=Flase)(i)
    X = Dense(Ty, activation='sigmoid')(X)
    model = Model(inputs=[i], outputs=X)
    
    return model

Using TensorFlow backend.


# LSTM model

In [3]:
Tx, Ty =  6, 1

LSTM_model = model((Tx,1),Ty)
print(LSTM_model.summary())
LSTM_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

LSTM_model.load_weights('sources/LSTM_sequence_weights.h5')

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 6, 1)              0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 16)                1152      
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 17        
Total params: 1,169
Trainable params: 1,169
Non-trainable params: 0
_________________________________________________________________
None


# GRU model

In [26]:
Tx, Ty =  6, 1

GRU_model = model_GRU((Tx,1),Ty)
#print(LSTM_model.summary())
GRU_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

GRU_model.load_weights('sources/GRU_sequence_weights.h5')


# Band centers

In [ ]:
center_min, center_max  = [759., 769.]
center_max += 0.5

In [ ]:
path_search = 'Search/'
band_centers_ini = [str(int(value*10)) for value in arange(center_min, center_max, 0.5)]
band_centers = ['.{}0000000E+03'.format(str(int(value*10))) for value in arange(center_min, center_max, 0.5)]

current_path = os.getcwd()

In [ ]:
os.chdir(current_path + path_search)
#print(os.getcwd())
calculate_RES_for_band_centers(band_centers)
os.chdir('../')

# Clear res file

In [2]:
res_path = 'Search/RES_files/'
for center in band_centers_ini:
    clear_res_file(res_path + 'TRANS_'+center+'.RES')

NameError: name 'band_centers_ini' is not defined

In [6]:
import os
import shutil 
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler


from utils import *


# Asym and search files

In [9]:
seria_name, quan_num = 'J1J-1', [1, 1, 0]
peak_file = 'expfile_st'

In [10]:
#os.chdir('../')
path_search = 'Search/'
current_path = os.getcwd()

res_file = 'Search/RES_files/TRANS_7640.RES'

shutil.copyfile(res_file,  os.getcwd()+'/' + path_search +'TRANS.RES')

os.chdir(current_path +'/' + path_search)
make_asym_search(peak_file, quan_num, max_counter=6)
deltas = deltas_in_search()
os.chdir('../')

# Make sequences and PREDICTION 

In [6]:
numb_seq = 1
for k,v in deltas.items():
    numb_seq *= len(deltas[k])
print(numb_seq/1000000,  'millions')

891.891 millions


In [7]:
max_numb = 10000

In [8]:
data_train = pd.read_csv('sources/train.csv')
X = data_train.drop(['Y'], axis='columns').values

ss = StandardScaler()
X = ss.fit_transform(X)


In [30]:
keys = list(deltas.keys())
ini_key = keys[0]
sequence = []
#data = []
counter = 0

def tree_2(k, seq_ini, counter=counter):
    for v in deltas[k]:
        sequence = seq_ini.copy()
        sequence.append(v)
        counter += 1
        if counter % 10000 == 0:
                print(counter)
        #if counter == max_numb:
        #    a=input()
        #    return 0
        if len(sequence) == len(deltas.items()):
            sequence_np = np.array([sequence])
            sequence_np = ss.transform(sequence_np)
            sequence_np = np.expand_dims(sequence_np, axis=2)
            prediction = LSTM_model.predict(sequence_np)

            if prediction > 0.6: 
                print(prediction, sequence)
                a=input()
            #print(sequence)
        else:
            counter = tree_2(k+1, sequence,counter=counter)
        if counter == 0:
            return 0
    return counter
        
tree_2(ini_key,sequence, counter=counter)

10000
[[0.6017649]] [-0.478, -0.496, -0.476, -0.141, -0.367, -0.4]

[[0.6026364]] [-0.478, -0.496, -0.476, -0.141, -0.367, -0.389]

[[0.6043055]] [-0.478, -0.496, -0.476, -0.141, -0.367, -0.367]


KeyboardInterrupt: 